# Causality Tutorial Exercises – Python

Contributors: Rune Christiansen, Jonas Peters, Niklas Pfister, Sorawit Saengkyongam, Sebastian Weichwald, Julius von Kügelgen.

The MIT License applies; copyright is with the authors.

Some exercises are adapted from "Elements of Causal Inference: Foundations and Learning Algorithms" by J. Peters, D. Janzing and B. Schölkopf.


# Exercise 1 – Structural Causal Model



Let's first draw a sample from an SCM

In [ ]:
import numpy as np

# set seed
np.random.seed(1)

rnorm = lambda n: np.random.normal(size=n)

n = 200
C = rnorm(n)
A = .8 * rnorm(n)
K = A + .1 * rnorm(n)
X = C - 2 * A + .2 * rnorm(n)
F = 3 * X + .8 * rnorm(n)
D = -2 * X + .5 * rnorm(n)
G = D + .5 * rnorm(n)
Y = 2 * K - D + .2 * rnorm(n)
H = .5 * Y + .1 * rnorm(n)

data = np.c_[C, A, K, X, F, D, G, Y, H]

__a)__

What are the parents and children of $X$ in the above SCM?

![DAG](https://raw.githubusercontent.com/sweichwald/causality-tutorial-exercises/main/data/Exercise-ANM.png)

**Answer:**
- $\mathbf{PA}_X=\{C,A\}$
- $\mathbf{CH}_X=\{F,D\}$


Take a pair of variables and think about whether you expect this pair to be dependent
(at this stage, you can only guess, later you will have tools to know). Check empirically.



Let's check for the pairs $(C,A)$ and $(K,X)$:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from scipy import stats

# Check correlation between C and A

plt.scatter(C, A)
plt.xlabel('C')
plt.ylabel('A')
plt.show()

stats.pearsonr(C, A)

In [ ]:
# Check correlation between K and X

plt.scatter(K, X)
plt.xlabel('K')
plt.ylabel('X')
plt.show()

stats.pearsonr(K, X)

__b)__

Generate a sample of size 300 from the interventional distribution $P_{\mathrm{do}(X=\mathcal{N}(2, 1))}$
and store the data matrix as `data_int`.

**Answer to b):**

To simulate the intervention, we modify the structural equation for $X$ and sample from the modified SCM.

In [ ]:
C_int = rnorm(n)
A_int = .8 * rnorm(n)
K_int = A_int + .1 * rnorm(n)
X_int = 2 + rnorm(n)  # this is the only change needed to model do(X=N(2,1))
F_int = 3 * X_int + .8 * rnorm(n)
D_int = -2 * X_int + .5 * rnorm(n)
G_int = D_int + .5 * rnorm(n)
Y_int = 2 * K_int - D_int + .2 * rnorm(n)
H_int = .5 * Y_int + .1 * rnorm(n)

data_int = np.c_[C_int, A_int, K_int, X_int, F_int, D_int, G_int, Y_int, H_int]

__c)__

Do you expect the marginal distribution of $Y$ to be different in both samples?

**Answer:**
In general, yes, because $Y$ is a descendant of the intervened node $X$.


__d)__

Do you expect the joint distribution of $(A, Y)$ to be different in both samples?


**Answer:** In general, yes: whereas the marginal of $A$ will stay the same, the conditional of $Y$ given $A$ will change since the intervention target $X$ lies on a path from $A$ to $Y$.

__e)__

Check your answers to c) and d) empirically.

In [ ]:
plt.hist(Y, density=True, alpha=0.5, label="observational")
plt.hist(Y_int, density=True, alpha=0.5, label="interventional")
plt.xlabel("Y")
plt.ylabel("density")
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.scatter(A, Y, alpha=0.5, label="observational")
plt.scatter(A_int, Y_int, alpha=0.5, label="interventional")
plt.xlabel("A")
plt.ylabel("Y")
plt.legend(loc='upper right')
plt.show()

# Exercise 2 – Adjusting


![DAG](https://raw.githubusercontent.com/sweichwald/causality-tutorial-exercises/main/data/Exercise-ANM.png)

Suppose we are given a fixed DAG (like the one above).

a) What are valid adjustment sets (VAS) used for?

**Answer:** To compute a correct (unbiased) estimate of causal effects.


b) Assume we want to find a VAS for the causal effect from $X$ to $Y$.
What are general recipies (plural 😉) for constructing VASs (no proof)?
Which sets are VAS in the DAG above?


**Answer:** (i) Parent adjustment via $\mathbf{PA}_X=\{C,A\}$. (ii) Backdoor adjustment, i.e., any set that blocks the backdoor path $X \leftarrow A \to K \to Y$, i.e., $\{A\}, \{A,K\}, \{K\}$; (iii) In general, any set containing at least one of $A$ or $K$, and not containing $D, G, H$.

c) The following code samples from an SCM. Perform linear regressions using different VAS and compare the regression coefficient against the causal effect from $X$ to $Y$.

In [ ]:
import numpy as np

# set seed
np.random.seed(1)

rnorm = lambda n: np.random.normal(size=n)

n = 200
C = rnorm(n)
A = .8 * rnorm(n)
K = A + .1 * rnorm(n)
X = C - 2 * A + .2 * rnorm(n)
F = 3 * X + .8 * rnorm(n)
D = -2 * X + .5 * rnorm(n)
G = D + .5 * rnorm(n)
Y = 2 * K - D + .2 * rnorm(n)
H = .5 * Y + .1 * rnorm(n)

data = np.c_[C, A, K, X, F, D, G, Y, H]

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.DataFrame(data, columns=['C', 'A', 'K', 'X', 'F', 'D', 'G', 'Y', 'H'])

# print(ols('Y ~ X', df).fit().params)
print(ols('Y ~ X + C + A + K', df).fit().params)
# print(ols('Y ~ X + K', df).fit().params)
# print(ols('Y ~ X + A + K', df).fit().params)
# print(ols('Y ~ X + F + K', df).fit().params)
print(ols('Y ~ X + C + A + K + F', df).fit().params)
print(ols('Y ~ X + A + K + F + G + H', df).fit().params)

**Answer:** All VAS yield a regression coefficient of approximately 2 which is the causal effect of $X$ on $Y$, obtained by multiplying the path coefficients along $X\to D \to Y$.

d) Why could it be interesting to have several options for choosing a VAS?

**Answer:**
- We have options to choose which variables to collect for estimating the causal effect
- If all variables are collected. We can use different adjustment sets to validate our assumptions, e.g., the validity of underlying graph or model misspecification.
- Even though all VASs should give the same result asymptoticallly, they may differ in terms of their finite-sample behaviour.


e) If you indeed have access to several VASs, what would you do?



**Answer:**
- Compare statistical efficiency, i.e., pick the estimator with the lowest variance.
- Choose the set of variables we are confident about the underlying graph and the data collection process.

# Exercise 3 – Independence-based Causal Structure Learning

__a)__

Assume $P^{X,Y,Z}$ is Markov and faithful wrt. $G$. Assume all (!) conditional independences are

$$
\newcommand{\indep}{{\,⫫\,}}
\newcommand{\dep}{\not{}\!\!\indep}
$$

$$X \indep Z \mid \emptyset$$

(plus symmetric statements). What is $G$?

**Answer:** This uniquely determines the graph as $X \to Y \leftarrow Z$.

__b)__ (optional; this one is more tricky)

Assume $P^{W,X,Y,Z}$ is Markov and faithful wrt. $G$. Assume all (!) conditional independences are

$$\begin{aligned}
(Y,Z) &\indep W \mid \emptyset \\
W &\indep Y \mid (X,Z) \\
(X,W) &\indep Y | Z
\end{aligned}
$$

(plus symmetric statements). What is $G$?

**Answer:** The graphs with edges $\{W,Z\}\to X$ and either (i) $Z\to Y$ or (ii)  $Y\to Z$.

Note: you can check your answers using this free online tool: https://www.dagitty.net/dags.html#

# Exercise 4 – Additive Noise Models

Set-up required packages:

In [ ]:
!mkdir ../data/
!wget https://raw.githubusercontent.com/sweichwald/causality-tutorial-exercises/main/data/Exercise-ANM.csv -q -O ../data/Exercise-ANM.csv
!wget https://raw.githubusercontent.com/sweichwald/causality-tutorial-exercises/main/python/kerpy/__init__.py -q -O kerpy.py
!pip install pygam

In [ ]:
from kerpy import hsic
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygam import GAM, s

Let's load and plot some real data set:

In [ ]:
data = pd.read_csv('../data/Exercise-ANM.csv')

plt.scatter(data["X"].values, data["Y"].values, s=2.);

__a)__

Do you believed that $X \to Y$ or that $X \gets Y$? Why?

**Answer:** $X \to Y$ because of the independent noise assumption: in the other direction, the noise would have to switch from bimodal to unimodal.

$$
\newcommand{\indep}{{\,⫫\,}}
\newcommand{\dep}{\not{}\!\!\indep}
$$

__b)__
Let us now try to get a more statistical answer. We have heard that we cannot
have  
$$Y = f(X) + N_Y,\ N_Y \indep X$$
and
$$X = g(Y) + N_X,\ N_X \indep Y$$
at the same time.

Given a data set over $(X,Y)$,
we now want to decide for one of the two models.

Come up with a method to do so.

Hints:
* `GAM(s(0)).fit(A, B).deviance_residuals(A, B)` provides residuals when regressing $B$ on $A$.
* `hsic(a, b)` can be used as an independence test (here, `a` and `b` are $n \times 1$ numpy arrays).

In [ ]:
X = data["X"].values
Y = data["Y"].values

# fitting GAM both directions
modelforw = GAM(s(0)).fit(X, Y)
modelbackw = GAM(s(0)).fit(Y, X)

# getting residuals for each model
resforw = modelforw.deviance_residuals(X, Y)
resbackw = modelbackw.deviance_residuals(Y, X)

# independence tests
pval_XY = hsic(resforw[:, np.newaxis], X[:, np.newaxis])
pval_YX = hsic(resbackw[:, np.newaxis], Y[:, np.newaxis])

print("p-values for rejecting the null hypothesis (independence of input and residuals)")
print("X->Y:", pval_XY)
print("Y->X:", pval_YX)

# creating plots
fig, axes = plt.subplots(2, 1, figsize=(7, 10))

axes[0].scatter(X, Y, s=2.)
axes[0].scatter(X, modelforw.predict(X))
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')

axes[1].scatter(Y,X, s=2.)
axes[1].scatter(Y, modelbackw.predict(Y))
axes[1].set_xlabel('Y')
axes[1].set_ylabel('X')

axes[0].set_title('X -> Y, pval: {:.4f}'.format(pval_XY))
axes[1].set_title('Y -> X, pval: {:.4f}'.format(pval_YX))

__c)__

Assume that the error terms are Gaussian with zero mean and variances
$\sigma_X^2$ and $\sigma_Y^2$, respectively.
The maximum likelihood for DAG G is
then proportional to
$-\log(\mathrm{var}(R^G_X)) - \log(\mathrm{var}(R^G_Y))$,
where $R^G_X$ and $R^G_Y$ are the residuals obtained from regressing $X$ and $Y$ on
their parents in $G$, respectively (no proof).

Find the maximum likelihood solution.

In [ ]:
# computing scores proportional to the likelihoods
print('X    Y: {:.2f}'.format(- np.log(np.var(X)) - np.log(np.var(Y)))) # X Y
print('X -> Y: {:.2f}'.format(- np.log(np.var(X)) - np.log(np.var(resforw)))) # X -> Y
print('Y -> X: {:.2f}'.format(- np.log(np.var(resbackw)) - np.log(np.var(Y)))) # X <- Y

# Exercise 5 – Invariant Causal Prediction

Set-up required packages and data:

In [ ]:
!mkdir ../data/
!wget https://raw.githubusercontent.com/sweichwald/causality-tutorial-exercises/main/data/Exercise-ICP.csv -q -O ../data/Exercise-ICP.csv

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

__a)__

Generate some observational and interventional data:

In [ ]:
# Generate n=1000 observations from the observational distribution
na = 1000
Xa = np.random.normal(size=na)
Ya = 1.5*Xa + np.random.normal(size=na)

# Generate n=1000 observations from an interventional distribution
nb = 1000
Xb = np.random.normal(loc=2, scale=1, size=nb)
Yb = 1.5*Xb + np.random.normal(size=nb)

# plot Y vs X1
fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(Xa, Ya, label='observational', marker='o', alpha=0.6)
ax.scatter(Xb, Yb, label='interventional', marker ='^', alpha=0.6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.legend();

Look at the above plot. Is the predictor $\{X\}$ an invariant set, that is (roughly speaking), does $Y \mid X = x$ have the same distribution in the orange and blue data?

**Answer:** Yes. Even though the marginal distribution of $X$ changes, the conditional distribution of $Y$ given $X=x$ appears to be the same for all $x$ across the two samples.

__b)__

We now consider data over a response and three covariates $X_1, X_2$, and $X_3$
and try to infer $\mathrm{pa}(Y)$. To do so, we need to find all sets for which this
invariance is satisfied.

In [ ]:
# load data
data = pd.read_csv('../data/Exercise-ICP.csv')
data['env'] = np.concatenate([np.repeat('observational', 140), np.repeat('interventional', 80)])
# pairplot
sns.pairplot(data, hue='env', height=2, plot_kws={'alpha':0.6});

In [ ]:
# The code below plots the residuals versus fitted values for all sets of
# predictors.
# extract response and predictors

Y = data['Y'].to_numpy()
X = data[['X1','X2','X3']].to_numpy()
# get environment indicator
obs_ind = data[data['env'] == 'observational'].index
int_ind = data[data['env'] == 'interventional'].index
# create all sets
all_sets = [(0,), (1,), (2,), (0,1), (0,2), (1,2), (0,1,2)]
# label each set
set_labels = ['X1', 'X2', 'X3', 'X1,X2', 'X1,X3', 'X2,X3', 'X1,X2,X3']

# fit OLS and store fitted values and residuals for each set
fitted = []
resid = []
for s in all_sets:
  model = sm.OLS(Y, X[:, s]).fit()
  fitted += [model.fittedvalues]
  resid += [model.resid]

# plotting function
def plot_fitted_resid(fv, res, ax, title):
  ax.scatter(fv[obs_ind], res[obs_ind], label='observational', marker='o', alpha=0.6)
  ax.scatter(fv[int_ind], res[int_ind], label='interventional', marker ='^', alpha=0.6)
  ax.legend()
  ax.set_xlabel('fitted values')
  ax.set_ylabel('residuals')
  ax.set_title(title)

# creating plots
fig, axes = plt.subplots(4, 2, figsize=(7,14))

# plot result for the empty set predictor
ax0 = axes[0,0]
ax0.scatter(obs_ind, Y[obs_ind], label='observational', marker='o', alpha=0.6)
ax0.scatter(int_ind, Y[int_ind], label='interventional', marker ='^', alpha=0.6)
ax0.legend()
ax0.set_xlabel('index')
ax0.set_ylabel('Y')
ax0.set_title('empty set')

# plot result for the other sets
for i, ax in enumerate(axes.flatten()[1:]):
  plot_fitted_resid(fitted[i], resid[i], ax, set_labels[i])

# make tight layout
plt.tight_layout()

Which of the sets are invariant? (There are two plots with four scatter plots each.)

**Answer:** $\{X_1, X_2, X_3\}$ and $\{X_1, X_2\}$ since the distribution of residuals for these sets appears to be the same across all domains.

__c)__
What is your best guess for $\mathrm{pa}(Y)$?

**Answer:** The intersection of the two sets, that is, $\{X_1, X_2\}$.

__d) (optional)__

Use the function ICP to check your result.

In [ ]:
!pip install causalicp

In [ ]:
import causalicp as icp

In [ ]:
# load data
raw = pd.read_csv('../data/Exercise-ICP.csv')
data = [raw[:140], raw[140:]]
icp.fit(data, 0, alpha=0.05, precompute=True, verbose=True, color=False)